In [1]:
# @title
pip install ipywidgets cirq

  Using cached cirq-1.3.0-py3-none-any.whl (8.1 kB)
  Using cached cirq_aqt-1.3.0-py3-none-any.whl (27 kB)
  Using cached cirq_core-1.3.0-py3-none-any.whl (1.8 MB)
  Using cached cirq_ft-1.3.0-py3-none-any.whl (143 kB)
  Using cached cirq_google-1.3.0-py3-none-any.whl (598 kB)
  Using cached cirq_ionq-1.3.0-py3-none-any.whl (60 kB)
  Using cached cirq_pasqal-1.3.0-py3-none-any.whl (31 kB)
  Using cached cirq_rigetti-1.3.0-py3-none-any.whl (66 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.8 MB/s eta 0:00:00
  Preparing metada

In [2]:
# @title
import ipywidgets as widgets

from IPython.display import display, clear_output


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cirq
import cirq_web

In [8]:
# @title
def alice_widgets():

  global bit_df

  # Section Title
  #===============
  title = widgets.HTML(value = "<h2 style='text-align: center;'>Alice</h2>")

  # Selecting which bit to encode
  #================================
  bit_selector = widgets.Dropdown(
    options=[('0', 0), ('1', 1), ('Choose a bit to encode', 2)],
    value=2,
    description='Bit:')

  # Displaying the encoded qubit based on the selected bit
  output_title = widgets.HTML(value = "<h3 style='text-align: center;'>Encoded Qubit</h2>")

  state_vector_output = widgets.Text(disabled = True)
  ket_output = widgets.Text(disabled = True)
  qubit_state_output = widgets.Output()

  def update_qubit_state(bit):

    global qubit, circuit

    if bit == 2:
      state_vector_output.value = ''
      ket_output.value = ''
      qubit_state_output.clear_output()

    else:

      qubit = cirq.NamedQubit('q0')
      circuit = cirq.Circuit()

      if bit == 0:
        circuit.append(cirq.I(qubit))
      else:
        circuit.append(cirq.H(qubit))

      state_vector = cirq.final_state_vector(circuit)
      ket = cirq.dirac_notation(state_vector = state_vector)

      state_vector_output.value = np.array2string(state_vector.real, precision = 2)
      ket_output.value = ket

      with qubit_state_output:
          clear_output(wait = True)
          display(cirq_web.bloch_sphere.BlochSphere(state_vector=state_vector, sphere_radius = 4))


  # Update the qubit state when the dropdown value changes
  bit_selector.observe(lambda change: update_qubit_state(change['new']), names='value')

  # Initialize with the default bit value
  #update_qubit_state(bit_selector.value)


  # Sending qubit
  #================
  send_button = widgets.Button(description = 'Send Qubit')
  send_button.style.button_color = '#98dcff'

  def on_click_send(b):

    if bit_df.iloc[2, -1] == 'not determined yet':
      display('Bob must measure the current qubit before Alice can send a new one.')

    elif bit_selector.value not in [0, 1]:
      display('Alice must choose a bit to encode first.')

    else:
      name = 'Bit ' + str(len(bit_df.columns) - 1)
      bit_df[name] = [str(bit_selector.value), 'not determined yet', 'not determined yet']
      update_bits()

      bit_selector.value = 2

      state_vector = cirq.final_state_vector(circuit)
      ket = cirq.dirac_notation(state_vector = state_vector)

      sections['Channel'].children[3].value = np.array2string(state_vector.real, precision = 2)
      sections['Channel'].children[4].value = ket

      with sections['Channel'].children[5]:
        clear_output(wait = True)
        display(cirq_web.bloch_sphere.BlochSphere(state_vector=state_vector, sphere_radius = 4))


  send_button.on_click(on_click_send)



  return [title, bit_selector, output_title, state_vector_output, ket_output, qubit_state_output, send_button]

In [90]:
# @title
def channel_widgets():

  global bit_df

  # Section Title
  #===============
  title = widgets.HTML(value = "<h2 style='text-align: center;'>Channel</h2>")

  # Displaying the encoded qubit based on the selected bit
  output_title = widgets.HTML(value = "<h3 style='text-align: center;'>Encoded Qubit</h2>")

  state_vector_output = widgets.Text(disabled = True)
  ket_output = widgets.Text(disabled = True)
  qubit_state_output = widgets.Output()


  # Eve intercepting
  #================================================
  eve_intercept_button = widgets.Button(description = 'Eve intercepts')

  def on_click_intercept(b):

    global bit_df, qubit, circuit

    if bit_df.iloc[1, -1] in ['0', '1']:
      display('Eve has already intercepted this qubit.')

    else:
      circuit.append(cirq.measure(qubit, key = 'Eve'))
      sim = cirq.Simulator()
      measurement = sim.run(circuit).measurements['Eve'][0][0]

      circuit = cirq.Circuit()

      if measurement == 0: circuit.append(cirq.I(qubit))
      else: circuit.append(cirq.H(qubit))

      state_vector = cirq.final_state_vector(circuit)
      ket = cirq.dirac_notation(state_vector = state_vector)

      state_vector_output.value = np.array2string(state_vector.real, precision = 2)
      ket_output.value = ket

      with qubit_state_output:
        clear_output(wait = True)
        display(cirq_web.bloch_sphere.BlochSphere(state_vector=state_vector, sphere_radius = 4))


      bit_df.iloc[1, -1] = str(measurement)
      update_bits()


  eve_intercept_button.on_click(on_click_intercept)


  # Sending qubit
  #================
  send_button = widgets.Button(description = 'Send Qubit')
  send_button.style.button_color = '#98dcff'

  def on_click_send(b):

    state_vector = cirq.final_state_vector(circuit)
    ket = cirq.dirac_notation(state_vector = state_vector)

    sections['Bob'].children[3].value = np.array2string(state_vector.real, precision = 2)
    sections['Bob'].children[4].value = ket

    with sections['Bob'].children[5]:
      clear_output(wait = True)
      display(cirq_web.bloch_sphere.BlochSphere(state_vector=state_vector, sphere_radius = 4))


    if bit_df.iloc[1, -1] == 'not determined yet':
      bit_df.iloc[1, -1] = 'Eve not intercepting'
      update_bits()

    state_vector_output.value = ''
    ket_output.value = ''

    qubit_state_output.clear_output()


  send_button.on_click(on_click_send)



  return [title, eve_intercept_button, output_title, state_vector_output, ket_output, qubit_state_output, send_button]

In [48]:
# @title
def bob_widgets():

  global bit_df

  # Section Title
  #===============
  title = widgets.HTML(value = "<h2 style='text-align: center;'>Bob</h2>")

  # Selecting which basis to use
  #================================
  basis_selector = widgets.Dropdown(
    options=[('0/1 (Z)', 0), ('+/- (H)', 1), ('Choose Bob\'s basis', 2)],
    value=2,
    description='Basis:')

  # Displaying the encoded qubit based on the selected bit
  output_title = widgets.HTML(value = "<h3 style='text-align: center;'>Received Qubit</h2>")

  state_vector_output = widgets.Text(disabled = True)
  ket_output = widgets.Text(disabled = True)
  qubit_state_output = widgets.Output()

  def update_qubit_state(basis):

    global qubit, circuit

    if basis != 2:

      # Make sure to remove any "temporary" guesses from Bob
      if len(circuit) > 1: del circuit[-1]

      if basis == 0:
        circuit.append(cirq.I(qubit))
      elif basis == 1:
        circuit.append(cirq.H(qubit))

      state_vector = cirq.final_state_vector(circuit)
      ket = cirq.dirac_notation(state_vector = state_vector)

      state_vector_output.value = np.array2string(state_vector.real, precision = 2)
      ket_output.value = ket

      with qubit_state_output:
          clear_output(wait = True)
          display(cirq_web.bloch_sphere.BlochSphere(state_vector=state_vector, sphere_radius = 4))


  # Update the qubit state when the dropdown value changes
  basis_selector.observe(lambda change: update_qubit_state(change['new']), names='value')


  # Measuring qubit
  #================
  measure_button = widgets.Button(description = 'Measure Qubit')
  measure_button.style.button_color = '#98dcff'

  def on_click_measure(b):

    if bit_df.iloc[2, -1] in ['0', '1']:
      display('Bob has already measured this qubit.')

    elif basis_selector.value not in [0, 1]:
      display('Bob must choose a basis first.')

    else:

      circuit.append(cirq.measure(qubit, key = 'Bob'))
      sim = cirq.Simulator()
      measurement = sim.run(circuit).measurements['Bob'][0][0]



      bit_df.iloc[2, -1] = str(measurement) + ' in ' + ['Z', 'H'][basis_selector.value] + ' basis'
      update_bits()


      state_vector_output.value = ''
      ket_output.value = ''
      qubit_state_output.clear_output()

      basis_selector.value = 2


  measure_button.on_click(on_click_measure)



  return [title, basis_selector, output_title, state_vector_output, ket_output, qubit_state_output, measure_button]

In [88]:
# @title
def bit_widgets():

  title = widgets.HTML(value = "<h2 style='text-align: center;'>Bits</h2>")
  bit_output = widgets.HTML(value = bit_df.to_html())


  determine_keys_button = widgets.Button(description = 'Determine Keys')
  determine_keys_button.style.button_color = '#98dcff'
  determine_keys_button.layout.margin = '100px 0px 0px 0px'

  key_output = widgets.HTML(value = key_df.to_html())

  def on_click_determine(b):

    global bit_df

    if bit_df.iloc[2, -1] == 'not determined yet':
      display('Bob must receive and measure the current qubit first.')

    else:

      for i in range(1, bit_df.shape[1]):
        bits = bit_df.iloc[:, i].values

        # Only keep the bits that Bob measured as 1,
        # since Bob and Alice can be certain they used
        # different bases.
        if '1' in bits[2]:

          # If Bob measured 1 in the Z basis (0/1)
          # Alice must have prepared the + state
          # So they can agree this bit is a 1.
          # Otherwise, it's a 0.
          # Currently, assuming Eve just keeps whatever her results were
          # and hopes for the best.
          if 'Z' in bits[2]: key_df['Bit ' + str(i - 1)] = [1, bit_df.iloc[1, i], 1]
          else: key_df['Bit ' + str(i - 1)] = [0, bit_df.iloc[1, i], 0]

        else:
          key_df['Bit ' + str(i - 1)] = ['not used', 'not used', 'not used']

      update_keys()


  determine_keys_button.on_click(on_click_determine)


  return [title, bit_output, determine_keys_button, key_output]


def update_bits(): sections['Bits'].children[1].value = bit_df.to_html(index = False)
def update_keys(): sections['Bits'].children[3].value = key_df.to_html(index = False)

In [92]:
# @title
bit_df = pd.DataFrame({'Person': ['Alice', 'Eve', 'Bob']})
key_df = pd.DataFrame({'Person': ['Alice', 'Eve', 'Bob']})
circuit = None
qubit = None

sections = {'Alice': alice_widgets(), 'Channel': channel_widgets(), 'Bob': bob_widgets(), 'Bits': bit_widgets()}
sections = {k: widgets.VBox(sections[k], layout=widgets.Layout(border='2px solid black', padding='5px', align_items='center')) for k in sections}


grid = widgets.GridBox(list(sections.values()), layout=widgets.Layout(grid_template_columns="repeat(4, 1fr)", grid_gap='10px', grid_auto_rows='750px'))
title = widgets.HTML("<h1 style='text-align: center; background-color: #012d9c; padding: 10px; color: #ffffff'>B92 Protocol</h1>")
display(widgets.VBox([title, grid]))

# End of Notebook

---
© 2024 The Coding School, All rights reserved